## Problem 1: Geocode shopping centers (5 points)

The overall aim of problems 1-3 is to find out **how many people live within a walking distance (1.5 km) from certain shopping centers in Helsinki**.

In problem 1 aim is to find out the addresses of shopping centers and to retreive numercal coordinates for these addresses. As an output, we will have a Shapefile called `shopping_centers.shp` which contains the geocoded result.

**Preparation:** Find out the addresses for following shopping centers from the internet, and write the addresses into a text file called `shopping_centers.txt`:

 - Itis
 - Forum
 - Iso-omena
 - Sello
 - Jumbo
 - REDI
 - Tripla 
 
 *Hint for "Tripla": This shopping center opened in October 10 2019, and if you are doing this exercise soon after that, the official address might not yet be in online databases. 
 Check for an address nearby the Pasila railway station on OpenStreetMap.org and use that as input.*

`shopping_centers.txt` should have semicolon (`;`) as a separator, and the file should include the following columns:

- ``id`` (integer) containing an unique identifier for each shopping center
- ``name`` (string) of each shopping center
- ``addr`` (string) the address 


See and example of how to format the text file [in the lesson 3 materials](https://automating-gis-processes.github.io/site/master/notebooks/L3/geocoding_in_geopandas.html). Save (and upload) the text file into your exercise repository.

- Read `shopping_centers.txt` that you just created into a pandas DataFrame called ``data``:

In [1]:
# Import modules
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

fp = "shopping_centers.txt"

# Read the data (replace "None" with your own code)
data = pd.read_csv(fp, sep=';')

In [2]:
#NON-EDITABLE TEST CELL
# Check your input data
print(data)

   id       name                                            addr
0   0       Itis            Itäkatu 1-7, 00930 Helsinki, Finland
1   1      Forum  Mannerheimintie 14-20, 00100 Helsinki, Finland
2   2  Iso-Omena           Piispansilta 11, 02230 Espoo, Finland
3   3      Sello       Leppävaarankatu 3-9, 02600 Espoo, Finland
4   4      Jumbo      Vantaanportinkatu 3, 01510 Vantaa, Finland
5   5       REDI   Hermannin rantatie 5, 00580 Helsinki, Finland
6   6     Tripla        Pasilansilta 11, 00520 Helsinki, Finland


- Geocode the addresses using the Nominatim geocoding service. Store the output in a variable called `geo`:

In [3]:
from geopandas.tools import geocode
# Geocode the addresses using Nominatim
geo = None

# REPLACE THE ERROR BELOW WITH YOUR OWN CODE
geo = geocode(data['addr'], provider = "nominatim", user_agent='practice', timeout=4)

In [4]:
#NON-EDITABLE TEST CELL
# Check the geocoded output
print(geo)

                    geometry  \
0  POINT (25.08294 60.21170)   
1  POINT (24.93828 60.16878)   
2  POINT (24.73995 60.16040)   
3  POINT (24.81042 60.21752)   
4  POINT (24.96591 60.29044)   
5  POINT (24.97904 60.18702)   
6  POINT (24.93433 60.19821)   

                                             address  
0  Kauppakeskus Itis, 1-7, Itäkatu, Itäkeskus, Va...  
1  Salaattiasema, 14-20, Mannerheimintie, Keskust...  
2  H&M, 11, Piispansilta, Matinkylä, Suur-Matinky...  
3  Lasten kappeli Arkki, 3-9, Leppävaarankatu, Sä...  
4  K-Citymarket Jumbo, 3, Vantaanportinkatu, Vant...  
5  Silta, 5, Hermannin rantatie, Verkkosaari, Kal...  
6  Pasilansilta, Keski-Pasila, Pasila, Keskinen s...  


In [5]:
#NON-EDITABLE TEST CELL
# Check the data type (should be a GeoDataFrame!)
print(type(geo))

<class 'geopandas.geodataframe.GeoDataFrame'>


In [6]:
geo.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

Check that the coordinate reference system of the geocoded result is correctly defined, and **reproject the layer into ETRS GK-25** (EPSG:3879):

In [7]:
geo = geo.to_crs(3879)

In [8]:
#NON-EDITABLE TEST CELL
# Check layer crs
print(geo.crs)

epsg:3879


- Make a table join between the geocoded addresses (``geo``) and the original addresses (``data``) in order to link the numerical coordinates and  the `id` and `name` of each shopping center. 
- Store the output in a variable called ``geodata`` 


In [9]:
# Join the tables
geodata = None

geodata = geo.join(data)

In [10]:
#NON-EDITABLE TEST CELL
# Check the join output
print(geodata.head())

                           geometry  \
0  POINT (25504598.602 6677662.109)   
1  POINT (25496573.542 6672878.360)   
2  POINT (25485559.440 6671971.772)   
3  POINT (25489491.076 6678322.265)   
4  POINT (25498114.526 6686432.573)   

                                             address  id       name  \
0  Kauppakeskus Itis, 1-7, Itäkatu, Itäkeskus, Va...   0       Itis   
1  Salaattiasema, 14-20, Mannerheimintie, Keskust...   1      Forum   
2  H&M, 11, Piispansilta, Matinkylä, Suur-Matinky...   2  Iso-Omena   
3  Lasten kappeli Arkki, 3-9, Leppävaarankatu, Sä...   3      Sello   
4  K-Citymarket Jumbo, 3, Vantaanportinkatu, Vant...   4      Jumbo   

                                             addr  
0            Itäkatu 1-7, 00930 Helsinki, Finland  
1  Mannerheimintie 14-20, 00100 Helsinki, Finland  
2           Piispansilta 11, 02230 Espoo, Finland  
3       Leppävaarankatu 3-9, 02600 Espoo, Finland  
4      Vantaanportinkatu 3, 01510 Vantaa, Finland  


- Save the output as a Shapefile called `shopping_centers.shp` 

In [11]:
# Define output filepath
out_fp = "shopping_centers.shp"

# Save file

geodata.to_file(out_fp)

In [12]:
#NON-EDITABLE TEST CELL
# Print info about output file
print("Geocoded output is stored in this file:", out_fp)

Geocoded output is stored in this file: shopping_centers.shp


## Problem 2: Create buffers around shopping centers (5 points)

Let's continue with our case study and calculate a 1.5 km buffer around the geocoded points. 


- Start by creating a new column called `buffer` to ``geodata`` GeoDataFrame:

In [13]:
geodata['buffer'] = None

- Calculate a 1.5 km buffer for each geocoded point. Store the buffer geometry in the new `buffer` column.

Here, you can use the [GeoDataFrame buffer() method](http://geopandas.org/geometric_manipulations.html#GeoSeries.buffer), which uses Shapely's [buffer](http://toblerity.org/shapely/manual.html#object.buffer) in the bacground. You only need to use the `distance` -parameter, don't worry about the other parameters.

In [14]:
# Buffer of 1500m
geodata['buffer'] = geodata.buffer(1500)

In [15]:
#NON-EDITABLE TEST CELL
print(geodata.head())

                           geometry  \
0  POINT (25504598.602 6677662.109)   
1  POINT (25496573.542 6672878.360)   
2  POINT (25485559.440 6671971.772)   
3  POINT (25489491.076 6678322.265)   
4  POINT (25498114.526 6686432.573)   

                                             address  id       name  \
0  Kauppakeskus Itis, 1-7, Itäkatu, Itäkeskus, Va...   0       Itis   
1  Salaattiasema, 14-20, Mannerheimintie, Keskust...   1      Forum   
2  H&M, 11, Piispansilta, Matinkylä, Suur-Matinky...   2  Iso-Omena   
3  Lasten kappeli Arkki, 3-9, Leppävaarankatu, Sä...   3      Sello   
4  K-Citymarket Jumbo, 3, Vantaanportinkatu, Vant...   4      Jumbo   

                                             addr  \
0            Itäkatu 1-7, 00930 Helsinki, Finland   
1  Mannerheimintie 14-20, 00100 Helsinki, Finland   
2           Piispansilta 11, 02230 Espoo, Finland   
3       Leppävaarankatu 3-9, 02600 Espoo, Finland   
4      Vantaanportinkatu 3, 01510 Vantaa, Finland   

                   

In [16]:
#NON-EDITABLE TEST CELL
# Check the data type of the first value in the buffer-column
print(type(geodata.at[0,'buffer']))

<class 'shapely.geometry.polygon.Polygon'>


In [17]:
#NON-EDITABLE TEST CELL
# Check the areas of your buffers in km^2
print(round(gpd.GeoSeries(geodata["buffer"]).area / 1000000))

0    7.0
1    7.0
2    7.0
3    7.0
4    7.0
5    7.0
6    7.0
dtype: float64


- Replace the values in `geometry` column with the values of `buffer` column:

In [18]:
geodata['geometry'] = geodata['buffer']

In [19]:
#NON-EDITABLE TEST CELL
print(geodata.head())

                                            geometry  \
0  POLYGON ((25506098.602 6677662.109, 25506091.3...   
1  POLYGON ((25498073.542 6672878.360, 25498066.3...   
2  POLYGON ((25487059.440 6671971.772, 25487052.2...   
3  POLYGON ((25490991.076 6678322.265, 25490983.8...   
4  POLYGON ((25499614.526 6686432.573, 25499607.3...   

                                             address  id       name  \
0  Kauppakeskus Itis, 1-7, Itäkatu, Itäkeskus, Va...   0       Itis   
1  Salaattiasema, 14-20, Mannerheimintie, Keskust...   1      Forum   
2  H&M, 11, Piispansilta, Matinkylä, Suur-Matinky...   2  Iso-Omena   
3  Lasten kappeli Arkki, 3-9, Leppävaarankatu, Sä...   3      Sello   
4  K-Citymarket Jumbo, 3, Vantaanportinkatu, Vant...   4      Jumbo   

                                             addr  \
0            Itäkatu 1-7, 00930 Helsinki, Finland   
1  Mannerheimintie 14-20, 00100 Helsinki, Finland   
2           Piispansilta 11, 02230 Espoo, Finland   
3       Leppävaarankatu 

Optional: at this point, you can drop out unnecessary columns from the geodataframe. In the next problem, 
we will only need these columns: `'id', 'name', 'geometry'`

In [20]:
# Dropping useless columns. Better way would have been selecting desired columns.
geodata = geodata.drop(columns=['addr', 'buffer', 'address'])

In [21]:
geodata.head()

,geometry,id,name
0,"POLYGON ((25506098.602 6677662.109, 25506091.3...",0,Itis
1,"POLYGON ((25498073.542 6672878.360, 25498066.3...",1,Forum
2,"POLYGON ((25487059.440 6671971.772, 25487052.2...",2,Iso-Omena
3,"POLYGON ((25490991.076 6678322.265, 25490983.8...",3,Sello
4,"POLYGON ((25499614.526 6686432.573, 25499607.3...",4,Jumbo


## Problem 3: How many people live near shopping centers? (5 points)

Last step in our analysis is to make a spatial join between our buffer layer and population data in order to find out **how many people live near each shopping center**. We will use the same data as we did during [lesson 3](https://automating-gis-processes.github.io/site/notebooks/L3/spatial-join.html#Spatial-join); **a Population Grid** that is available via the HSY wfs. 

Alternatively, you can also download the data from the [Helsinki Region Infoshare (HRI)](https://www.hsy.fi/fi/asiantuntijalle/avoindata/Sivut/AvoinData.aspx?dataID=7) as a shapefile (using wget).

The coordinate reference system of the population grid is **ETRS GK-25 (EPSG:3879)**.


**Steps:**

- Read the population grid into a geodataframe

- Select only the useful columns from the population grid: ``'asukkaita'`` (=population count per grid square) and ``'geometry'`` 

- Make a spatial join between your buffered point layer and population grid layer. Join the information now from buffer layer **into the population grid layer**

- Group the joined layer by shopping center index

- Calculate the sum of population living within 1.5 km for each shopping center.

**Finally:**

- Print out the population living within 1.5 km from each shopping center:

     - Itis
     - Forum
     - Iso-omena
     - Sello
     - Jumbo
     - REDI
     - Tripla
     
**Final print out should contain both the shopping center name and population count**, for example: `26532 people 
live within 1.5 km from Iso-Omena`.

*Feel free to divide your solution into several codeblocks! Remember to comment your code  :)*

In [36]:
# From course tutorial:
# Reading population data for 2021 from: https://hri.fi/data/en_GB/dataset/vaestotietoruudukko. 
from pyproj import CRS
import requests
import geojson


url = 'https://kartta.hsy.fi/geoserver/wfs'
params = dict(service='WFS',
              version='2.0.0',
              request='GetFeature',
              typeName='asuminen_ja_maankaytto:Vaestotietoruudukko_2021',
              outputFormat='json')
r = requests.get(url, params=params)

pop = gpd.GeoDataFrame.from_features(geojson.loads(r.content))


In [37]:
# Checked the crs from metadata
pop = pop.set_crs(3879)

In [38]:
# renaming asukkaita as pop21 and creating subselection
pop = pop.rename(columns={'asukkaita' : 'pop21'})
pop.columns
pop = pop[['pop21', 'geometry']]

In [39]:
#NON-EDITABLE TEST CELL
# Check your input data
print("Number of rows:", len(pop))
print(pop.head(3))

Number of rows: 5853
   pop21                                           geometry
0      5  POLYGON ((25472499.995 6689749.005, 25472499.9...
1      7  POLYGON ((25472499.995 6685998.998, 25472499.9...
2      8  POLYGON ((25472499.995 6684249.004, 25472499.9...


In [41]:
pop.crs == geo.crs

True

In [42]:
# Creating a spatial join between grid layer and buffer layer. 
join_dp = gpd.sjoin(pop,geodata, how= "inner", predicate="within")
join_dp

,pop21,geometry,index_right,id,name
1147,76,"POLYGON ((25484250.000 6672249.006, 25484250.0...",2,2,Iso-Omena
1148,19,"POLYGON ((25484250.000 6671748.997, 25484250.0...",2,2,Iso-Omena
1209,108,"POLYGON ((25484499.997 6672749.004, 25484499.9...",2,2,Iso-Omena
1210,133,"POLYGON ((25484499.997 6672499.005, 25484499.9...",2,2,Iso-Omena
1211,71,"POLYGON ((25484499.997 6672249.006, 25484499.9...",2,2,Iso-Omena
...,...,...,...,...,...
5308,350,"POLYGON ((25505499.997 6677498.998, 25505499.9...",0,0,Itis
5309,362,"POLYGON ((25505499.997 6677248.998, 25505499.9...",0,0,Itis
5362,358,"POLYGON ((25505749.995 6677748.997, 25505749.9...",0,0,Itis
5363,128,"POLYGON ((25505749.995 6677498.998, 25505749.9...",0,0,Itis


In [44]:
grouped = join_dp.groupby('id')

In [46]:
# Report how many people live within 1.5 km distance from each shopping center
# Creating lists for id and population and appending the values to lists
total_pop = 0
populations = []
pop = pd.DataFrame()
pop['id'] = None
pop['population'] = None
for key,group in grouped:
    for row in group['pop21']:
        total_pop = total_pop + row
    pop.loc[key, 'id'] = key
    pop.loc[key, 'population'] = total_pop
    total_pop = 0
pop

,id,population
0,0,20680
1,1,56296
2,2,26295
3,3,22984
4,4,11715
5,5,29085
6,6,24815


In [47]:
# Merging the population data on "id" which was saved on previous step to geodata
pop_join = geodata.merge(pop, on='id')

In [48]:
pop_join

,geometry,id,name,population
0,"POLYGON ((25506098.602 6677662.109, 25506091.3...",0,Itis,20680
1,"POLYGON ((25498073.542 6672878.360, 25498066.3...",1,Forum,56296
2,"POLYGON ((25487059.440 6671971.772, 25487052.2...",2,Iso-Omena,26295
3,"POLYGON ((25490991.076 6678322.265, 25490983.8...",3,Sello,22984
4,"POLYGON ((25499614.526 6686432.573, 25499607.3...",4,Jumbo,11715
5,"POLYGON ((25500337.156 6674909.983, 25500329.9...",5,REDI,29085
6,"POLYGON ((25497857.756 6676158.372, 25497850.5...",6,Tripla,24815


In [49]:
for i,j in zip(pop_join['population'],pop_join['name']):
    print(i, "people live within 1.5 km from", j)

20680 people live within 1.5 km from Itis
56296 people live within 1.5 km from Forum
26295 people live within 1.5 km from Iso-Omena
22984 people live within 1.5 km from Sello
11715 people live within 1.5 km from Jumbo
29085 people live within 1.5 km from REDI
24815 people live within 1.5 km from Tripla


**Reflections:**
    
- How challenging did you find problems 1-3 (on scale to 1-5), and why?
- What was easy?
- What was difficult?

I found these exercises easy (2) since there were very good instrcutions and the join/spatial join made sense for me. I don't have experience with geocoding and getting data from WFS so that I have to practise.

Well done! Now you can continue to [problem 4](Exercise-3-Problem-4.ipynb)